<a href="https://colab.research.google.com/github/Matthew-Z-Purvis/Satellites/blob/main/Sat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from PIL import Image
import random
import shutil
from collections import defaultdict
import numpy as np

def clear_directory(directory):
    """
    Delete all files and directories inside the given directory.

    :param directory: The directory to clear.
    """
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)

def random_transform(image, original_size):
    """
    Apply random transformations to the image such as stretching, shrinking, and cropping.

    :param image: The image to transform.
    :param original_size: A tuple (width, height) of the original size.
    :return: The transformed image.
    """
    original_width, original_height = original_size

    # Ensure the new dimensions are at least the size of the original dimensions
    new_width = random.randint(original_width, int(original_width * 1.6))
    new_height = random.randint(original_height, int(original_height * 1.6))
    image = image.resize((new_width, new_height), Image.LANCZOS)

    # Random cropping to original size
    left = random.randint(0, max(0, new_width - original_width))
    top = random.randint(0, max(0, new_height - original_height))
    right = left + original_width
    bottom = top + original_height
    image = image.crop((left, top, right, bottom))

    return image

def is_yellow(pixel, threshold=200):
    """
    Check if a pixel is yellow based on a threshold.

    :param pixel: The pixel to check (R, G, B).
    :param threshold: The threshold for determining yellow.
    :return: True if the pixel is yellow, otherwise False.
    """
    r, g, b = pixel
    return r > 200 and g > 200 and b < threshold

def make_non_yellow_and_adjacent_black(image):
    """
    Make all non-yellow and non-adjacent-to-yellow pixels black in the image.

    :param image: The image to process.
    :return: The processed image.
    """
    np_image = np.array(image)
    yellow_mask = np.apply_along_axis(lambda pixel: is_yellow(pixel), 2, np_image)

    # Create a mask for pixels adjacent to yellow pixels
    adjacent_mask = np.zeros_like(yellow_mask)

    # Dilate the yellow mask to include adjacent pixels
    for y in range(1, yellow_mask.shape[0] - 1):
        for x in range(1, yellow_mask.shape[1] - 1):
            if yellow_mask[y, x]:
                adjacent_mask[y-1:y+2, x-1:x+2] = True


    combined_mask = yellow_mask | adjacent_mask


    np_image[~combined_mask] = [0, 0, 0]

    return Image.fromarray(np_image)

def crop_top_down_until_yellow(image, crop_box):
    """
    Crop the image from the top down until a yellow pixel is encountered.

    :param image: The image to crop.
    :param crop_box: The initial crop box.
    :return: The cropped image.
    """
    left, upper, right, lower = crop_box
    cropped_img = image.crop(crop_box)

    for y in range(upper, lower):
        row = cropped_img.crop((0, y - upper, right - left, y - upper + 1))
        for x in range(row.width):
            if is_yellow(row.getpixel((x, 0))):
                return image.crop((left, y, right, lower))
    return cropped_img

def process_and_resize_images(input_directory, output_directory, crop_box, resize_to, train_ratio=0.8):
    """
    Process and resize all images in the input directory and save them to the train and test directories.
    Additionally, augment each processed image with random transformations.

    :param input_directory: Directory containing the images to be processed and resized.
    :param output_directory: Directory to save the processed and resized images.
    :param crop_box: A tuple (left, upper, right, lower) defining the crop rectangle.
    :param resize_to: A tuple (width, height) defining the size to resize the images to.
    :param train_ratio: The ratio of images to be used for training (default is 0.8).
    """

    clear_directory(output_directory)


    train_directory = os.path.join(output_directory, "train")
    test_directory = os.path.join(output_directory, "test")
    os.makedirs(train_directory)
    os.makedirs(test_directory)


    class_images = defaultdict(list)
    for filename in os.listdir(input_directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
            class_name = filename[:5]
            class_images[class_name].append(filename)


    min_train_images = min([len(images) for images in class_images.values()]) * train_ratio
    min_test_images = min([len(images) for images in class_images.values()]) * (1 - train_ratio)

    for class_name, images in class_images.items():
        random.shuffle(images)
        train_images = images[:int(min_train_images)]
        test_images = images[int(min_train_images):int(min_train_images + min_test_images)]

        for image_set, subset_directory in zip([train_images, test_images], [train_directory, test_directory]):
            class_directory = os.path.join(subset_directory, class_name)
            os.makedirs(class_directory, exist_ok=True)

            for filename in image_set:
                img_path = os.path.join(input_directory, filename)
                with Image.open(img_path) as img:
                    processed_img = make_non_yellow_and_adjacent_black(img)
                    cropped_img = crop_top_down_until_yellow(processed_img, crop_box)
                    resized_img = cropped_img.resize(resize_to, Image.LANCZOS)

                    resized_img_path = os.path.join(class_directory, filename)
                    resized_img.save(resized_img_path)
                    print(f"Processed, cropped, and resized image saved as {resized_img_path}")

                    augment_times = 15
                    for i in range(augment_times):
                        augmented_img = random_transform(resized_img, resize_to)
                        augmented_img_path = os.path.join(class_directory, f"aug{i}_{filename}")
                        augmented_img.save(augmented_img_path)
                        print(f"Augmented image saved as {augmented_img_path}")


input_directory = "screenshots"
output_directory = "cropped_images"

crop_box = (180, 200, 665, 530)
resize_to = (256, 256)

process_and_resize_images(input_directory, output_directory, crop_box, resize_to)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os


base_dir = 'cropped_images'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.5,
    height_shift_range=0.5,
    shear_range=0.5,
    zoom_range=0.8,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=64,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=64,
    class_mode='categorical'
)

# Load the VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

base_model.trainable = False

# Custom layers
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 for seven satellites
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Learning rate scheduler
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)


# Training
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=test_generator,
    callbacks=[reduce_lr, early_stopping]
)




In [ ]:
#model.save('my_model.keras')


In [ ]:
#from tensorflow.keras.models import load_model

#model = load_model('Satmodel.keras')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator


base_dir = 'cropped_images'
test_dir = os.path.join(base_dir, 'test')


img_width, img_height = 256, 256
batch_size = 64


test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Important to ensure consistency
)


test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test accuracy: {test_acc}')


pred_probs = model.predict(test_generator, steps=len(test_generator))


pred_classes = np.argmax(pred_probs, axis=1)

true_classes = test_generator.classes

class_labels = list(test_generator.class_indices.keys())


cm = confusion_matrix(true_classes, pred_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

def predict_image_class(image_path, model, class_labels, crop_box, resize_to):
    img = Image.open(image_path)
    processed_img = make_non_yellow_and_adjacent_black(img)
    resized_img = processed_img.resize(resize_to, Image.LANCZOS)
    img_array = img_to_array(resized_img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    predicted_label = class_labels[predicted_class]
    return predicted_label


crop_box = (180, 200, 665, 530)
resize_to = (256, 256)


# Directory containing the test images
test_images_directory = 'testScreenshots'
acc = 0
count = 0

# Predict and print results for each image in the test directory
for filename in os.listdir(test_images_directory):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
        image_path = os.path.join(test_images_directory, filename)
        actual_class = filename[:5]  # Adjust based on the actual class naming convention
        predicted_label = predict_image_class(image_path, model, class_labels, crop_box, resize_to)
        print(f'Image: {filename}, Actual Class: {actual_class}, Predicted Class: {predicted_label}')
        if actual_class == predicted_label:
            acc += 1
        count += 1

print(f"Accuracy: {acc/count:.2f}")